In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile
data_dir='/content/drive/MyDrive/ColabNotebooks/refcocog.tar.gz'
# Extract data
tar = tarfile.open(data_dir)
tar.extractall('dataset/')

!tar -xf dataset/refcocog.tar.gz -C dataset
!ls dataset

In [ ]:
!pip install torch
!pip install torchvision
!pip install SummaryWriter
!pip install tensorboard
!pip install scikit-image
!pip install matplotlib
!pip install Lambda
!pip install opencv-python

!python -m pip install pyyaml==5.1
import sys, os, distutils.core



import torch
torch.cuda.current_device()
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import os
from posixpath import split
import json
import tarfile
import io
import pickle
import sys
from PIL import Image
from collections import OrderedDict
from pathlib import Path
from collections import defaultdict


import numpy as np
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.utils import draw_bounding_boxes
import torch
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import albumentations as A


# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

device = "cuda" if torch.cuda.is_available() else "cpu"
temperature = 1.0
torch.autograd.set_detect_anomaly(True)
data = {}
to_pil = T.ToPILImage()

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!git clone https://github.com/ultralytics/yolov5
!cd yolov5
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt
! cd ..

In [ ]:
from clip import clip

model, preprocess = clip.load("RN50")
model = model.float()
detector = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
class RefCOCOgDataset(Dataset):
    def __init__(self, dataset, transform, target_transform, data_dir='dataset/refcocog'):
        super(RefCOCOgDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform
        self.dataset = dataset



    def __getitem__(self, idx):
        data_item = {}
        fname = os.path.join(self.data_dir+'/images/', self.dataset[idx]['image'])
        #fname = os.path.join(self.data_dir+'/images/', list(self.dataset.items())[idx][0])
        image = Image.open(fname).convert('RGB')
        #texts = list(self.dataset.items())[idx][1]
        if self.transform:
            data_item['image'] = self.dataset[idx]['image']
        if self.target_transform:
            data_item['captions'] = clip.tokenize([sent for desc in self.dataset[idx]['captions'] for sent in desc])
        data_item['bbox'] = torch.Tensor(self.dataset[idx]['bbox'])
        return data_item


    def __len__(self):
        return len(self.dataset)



In [ ]:
def get_datasets(refs, train = True):
  ref_data = []
  for val in refs:
    if train:
      if val['split'] == 'train' or val['split'] == 'val':
        ref_data.append(val)
    else:
      if val['split'] == 'test':
        ref_data.append(val)

  return ref_data


def preProcess_datasets(data_dir, train):
  dataset = {}
  refs = {}
  f = open(f'{data_dir}/annotations/refs(umd).p', 'rb')
  data['refs'] = pickle.load(f)
  if train:
    refs = get_datasets(data['refs'], train=True)
  else:
    refs = get_datasets(data['refs'], train=False)
  instances_file = os.path.join(f'{data_dir}/annotations/instances.json')
  instances = json.load(open(instances_file, 'r'))
  data['images'] = instances['images']
  data['annotations'] = instances['annotations']
  print(data['refs'][0])
  print(data['images'][0])
  print(data['annotations'][0])
  count=0
  image_features = []
  for key,val in enumerate(refs):
    for v in data['images']:
      if(val['image_id'] == v['id']):
        fname = os.path.join(data_dir+'/images/', v['file_name'])
        if os.path.exists(fname):
          dataset[key] = {}
          img = Image.open(fname).convert('RGB')
          results = detector(img, size=640)
          results = non_max_suppression(results.pandas().xyxy[0], 0.2, 0.2)
          for det in results:
            for *xyxy, conf, cls in det:
              try:
                t = modified_model.givemebox(img, xyxy)
              except Exception as e:
                continue
                image_features /= image_features.norm(dim=-1, keepdim=True)
              im_features.append(image_features)
          im_features = torch.stack(im_features)
          dataset[key]['image'] = im_features
          dataset[key]['captions'] = [t['raw'] for t in val['sentences']]
          dataset[key]['id'] = val['image_id']
          break
    for c in instances['annotations']:
      if val['image_id']==c['image_id']:
          dataset[key]['bbox'] = c['bbox']
          break
    count=count+1
    if count==100:
      break

  return dataset


  def visualise_result(img, txt, bbox):
    resize_image = T.Resize(100)
    img = resize_image(img)
    convert_tensor = T.ToTensor()
    image = convert_tensor(img).to(device)
    image = image.clone().detach()
    image = image.type(torch.ByteTensor).to(device)
    boxs = torch.tensor(bbox,dtype=torch.int).to(device)
    boxes = boxs.reshape([1,4])
    boxes = torchvision.ops.box_convert(boxes, "cxcywh", "xyxy").to(device)
    image = draw_bounding_boxes(image=image, boxes=boxes, width=2, colors=(0,0,255), fill=True).to(device)
    image = image.permute(1,2,0)

In [8]:
import numpy as np

def calculate_precision_recall(pred_boxes, true_boxes, iou_threshold=0.5):
    sorted_indices = np.argsort(pred_boxes[:, -1])[::-1]
    pred_boxes = pred_boxes[sorted_indices]

    num_predictions = len(pred_boxes)
    num_targets = len(true_boxes)

    true_positives = np.zeros(num_predictions)
    false_positives = np.zeros(num_predictions)
    false_negatives = np.zeros(num_targets)

    for i, pred_box in enumerate(pred_boxes):
        best_iou = 0
        best_match_index = -1

        for j, true_box in enumerate(true_boxes):
            iou = calculate_iou1d(pred_box, true_box)

            if iou > best_iou and iou >= iou_threshold:
                best_iou = iou
                best_match_index = j

        if best_match_index >= 0:
            true_positives[i] = 1
            false_negatives[best_match_index] = 1
        else:
            false_positives[i] = 1

    cum_true_positives = np.cumsum(true_positives)
    cum_false_positives = np.cumsum(false_positives)
    cum_false_negatives = np.cumsum(false_negatives)

    precision = cum_true_positives / (cum_true_positives + cum_false_positives)
    recall = cum_true_positives / (cum_true_positives + cum_false_negatives)

    return precision, recall

def calculate_ap(precision, recall):
    recall = np.concatenate(([0], recall, [1]))
    precision = np.concatenate(([0], precision, [0]))

    for i in range(precision.size - 1, 0, -1):
        precision[i - 1] = np.maximum(precision[i - 1], precision[i])

    indices = np.where(recall[1:] != recall[:-1])[0] + 1
    ap = np.sum((recall[indices] - recall[indices - 1]) * precision[indices])

    return ap

def calculate_mAP(ap_values):
    mAP = np.mean(ap_values)
    return mAP

In [9]:
def calculate_iou1d(box1, box2):
    # Calculate coordinates of intersection rectangle
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Compute area of intersection
    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # Compute areas of the two bounding boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # Compute IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou


In [188]:
import numpy as np
import pandas as pd

def non_max_suppression(yolo_output, confidence_threshold, iou_threshold):
    # Extract bounding box coordinates, class predictions, and confidence scores from the YOLO output

    boxes = yolo_output.iloc[:,:4]
    class_predictions = yolo_output.iloc[:,6]
    confidence_scores = yolo_output.iloc[:,5]
    selected_boxes = []
    zipped = list(zip(boxes, class_predictions, confidence_scores))

    df = pd.DataFrame(zipped)
    for box, class_probs, confidence_scores in df.iterrows():
        # Apply confidence thresholding
        mask = confidence_scores >= confidence_threshold

        class_probs = class_probs[mask]
        confidence_scores = confidence_scores[mask]


        # Apply non-maximum suppression
        selected_indices = []
        while len(confidence_scores) > 0:
            # Select the box with the highest confidence score
            best_index = np.argmax(confidence_scores)
            selected_indices.append(best_index)

            # Calculate IoU between the best box and the rest
            ious = calculate_iou(box[best_index], box[selected_indices[:-1]])

            # Remove indices of boxes with IoU greater than the threshold
            indices_to_remove = [i for i, iou in enumerate(ious) if iou > iou_threshold]
            box = np.delete(box, indices_to_remove, axis=0)
            class_probs = np.delete(class_probs, indices_to_remove, axis=0)
            confidence_scores = np.delete(confidence_scores.cpu(), indices_to_remove, axis=0)

        # Collect the selected boxes along with class IDs and confidence scores
        selected_boxes.extend([(b[0], b[1], b[2], b[3], np.argmax(c), s) for b, c, s in zip(box, class_probs, confidence_scores)])

    return selected_boxes

def calculate_iou(box1, boxes2):
    x1 = np.maximum(box1[0], boxes2[0])
    y1 = np.maximum(box1[1], boxes2[1])
    x2 = np.minimum(box1[2], boxes2[2])
    y2 = np.minimum(box1[3], boxes2[3])

    intersection = np.maximum(0, x2 - x1 + 1) * np.maximum(0, y2 - y1 + 1)
    area_box1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area_boxes2 = (boxes2[:, 2] - boxes2[:, 0] + 1) * (boxes2[:, 3] - boxes2[:, 1] + 1)
    union = area_box1 + area_boxes2 - intersection

    iou = intersection / union
    return iou


In [11]:
def cosine_similarity(tensor1: torch.Tensor, tensor2: torch.Tensor):
  # normalise the image and the text

  tensor1 /= tensor1.norm(dim=-1, keepdim=True)
  tensor2 /= tensor2.norm(dim=-1, keepdim=True)

  # evaluate the cosine similarity between the sets of features
  cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
  similarity = cos(tensor1, tensor2)
  #similarity = (tensor1 @ tensor2.T)

  return similarity.to(device)

In [12]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [13]:
def get_optimizer(model, lr, wd, momentum):
  optimizer = torch.optim.SGD([
      {'params': model.classifier.parameters(), 'lr': lr}
  ], lr=lr / 10, weight_decay=wd, momentum=momentum)

  return optimizer

In [14]:
"""
Applies Batch Normalization over a 1D input (or 2D tensor)

Shape:
  Input: (N, C)
  Output: (N, C)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm1d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm1d(4, affine=False)
  >>> input = torch.rand(10, 4)
  >>> out = bn(input)
"""

class BatchNorm1d(torch.nn.Module):
  def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
    super().__init__()

    self.in_features = in_features
    self.track_running_stats = track_running_stats
    self.affine = affine
    self.momentum = momentum

    if self.affine:
      self.gamma = torch.nn.Parameter(torch.ones(self.in_features, 1))
      self.beta = torch.nn.Parameter(torch.zeros(self.in_features, 1))

    if self.track_running_stats:
      # register_buffer registers a tensor as a buffer that will be saved as part of the model
      # but which does not require to be trained, differently from nn.Parameter
      self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
      self.register_buffer('running_std', torch.ones(self.in_features, 1))

  def forward(self, x):
    # transpose (N, C) to (C, N)
    x = x.transpose(0, 1).contiguous().view(x.shape[1], -1)

    # calculate batch mean
    mean = x.mean(dim=1).view(-1, 1)

    # calculate batch std
    std = x.std(dim=1).view(-1, 1)

    # during training keep running statistics (moving average of mean and std)
    if self.training and self.track_running_stats:
      # no computational graph is necessary to be built for this computation
      with torch.no_grad():
        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
        self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std

    # during inference time
    if not self.training and self.track_running_stats:
      mean = self.running_mean
      std = self.running_std

    # normalize the input activations
    x = (x - mean) / std

    # scale and shift the normalized activations
    if self.affine:
      x = x * self.gamma + self.beta

    return x.transpose(0, 1)

In [15]:
"""
Applies Batch Normalization over a 2D or 3D input (4D tensor)

Shape:
  Input: (N, C, H, W)
  Output: (N, C, H, W)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm2d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm2d(4, affine=False)
  >>> input = torch.rand(10, 4, 5, 5)
  >>> out = bn(input)
"""

class BatchNorm2d(torch.nn.Module):
  def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
    super().__init__()

    self.in_features = in_features
    self.track_running_stats = track_running_stats
    self.affine = affine
    self.momentum = momentum

    if self.affine:
      self.gamma = torch.nn.Parameter(torch.ones(self.in_features, 1))
      self.beta = torch.nn.Parameter(torch.zeros(self.in_features, 1))

    if self.track_running_stats:
      # register_buffer registers a tensor as a buffer that will be saved as part of the model
      # but which does not require to be trained, differently from nn.Parameter
      self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
      self.register_buffer('running_std', torch.ones(self.in_features, 1))

  def forward(self, x):
    # transpose (N, C, H, W) to (C, N, H, W)
    x = x.transpose(0, 1)

    # store the shape
    c, bs, h, w = x.shape

    # collapse all dimensions except the 'channel' dimension
    x = x.contiguous().view(c, -1)

    # calculate batch mean
    mean = x.mean(dim=1).view(-1, 1)

    # calculate batch std
    std = x.std(dim=1).view(-1, 1)

    # keep running statistics (moving average of mean and std)
    if self.training and self.track_running_stats:
      with torch.no_grad():
        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
        self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std

    # during inference time
    if not self.training and self.track_running_stats:
      mean = self.running_mean
      std = self.running_std

    # normalize the input activations
    x = (x - mean) / std

    # scale and shift the normalized activations
    if self.affine:
      x = x * self.gamma + self.beta

    return x.view(c, bs, h, w).transpose(0, 1)


In [16]:
def training_step(net, data_loader, optimizer, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # set the network to training mode
    net = net.float()
    net.train()

  # iterate over the training set
    for batch_idx, batch in enumerate(data_loader):
      img = batch['image']
      bbox = batch['bbox']
      label = batch['captions']
      # forward pass
      outputs = net(batch)

      # loss computation
      loss = cost_function(outputs, bbox)
      #print('loss')
      #print(loss)
      # backward pass
      loss.backward()

      # parameters update
      optimizer.step()

      # gradients reset
      optimizer.zero_grad()

      # fetch prediction and loss value
      #samples += batch['image'].shape[0]'

      loss = cost_function(outputs, bbox)
      samples += img.shape[0]
      cumulative_loss += loss
      cIoU = calculate_iou(outputs, bbox)
      cumulative_cIoU += cIoU.sum().item()
      cumulative_count += len(outputs)
      # print(cumulative_cIoU)
      # print(cumulative_count)
      torch.cuda.empty_cache()
      samples += len(outputs)
      cumulative_loss += loss.item()

      cIoU = calculate_iou(outputs, bbox)
      cumulative_cIoU += cIoU.sum().item()
      cumulative_count += len(outputs)
      # print(cumulative_cIoU)
      # print(cumulative_count)
      torch.cuda.empty_cache()
    return cumulative_loss / samples, cumulative_cIoU / cumulative_count

def test_step(net, data_loader, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    net = net.float()
    # set the network to evaluation mode
    net.eval()

    with torch.no_grad():
      for batch_idx, batch in enumerate(data_loader):
        img = batch['image']
        bbox = batch['bbox']
        label = batch['captions']
        outputs = net(batch)
        loss = cost_function(outputs, bbox)

        samples += img.shape[0]
        cumulative_loss += loss

        cIoU = calculate_iou(outputs, bbox)
        cumulative_cIoU += cIoU.sum().item()
        cumulative_count += len(outputs)
        # print(cumulative_cIoU)
        # print(cumulative_count)
        torch.cuda.empty_cache()
    return cumulative_loss / samples, cumulative_cIoU / cumulative_count

In [17]:
class ProjectionHead(torch.nn.Module):
    def __init__(
        self,
        embedding_dim,
        in_features = 1024,
        out_features = 1024,
        bias=False
    ):
        super().__init__()
        self.classifier = torch.nn.Sequential(
          torch.nn.Linear(in_features, out_features, bias=bias),
          torch.nn.GELU(),
          torch.nn.Linear(out_features, out_features, bias=bias),
          torch.nn.Dropout(0.1),
          torch.nn.BatchNorm1d(1024),
        )

    def forward(self, x):
        projected = self.classifier(x)

        return x

In [18]:
class CustomCLIP(torch.nn.Module):
  def __init__(self, num_classes: int = 10, bias=False):
    super().__init__()
    self.detector = detector
    self.model = model
    in_features = 1024
    out_features = 1024
    embedding_dim = ''
    # take the visual encoder of CLIP
    # we also convert it to be 32 bit (by default CLIP is 16)
    self.encoder = model.visual.float()
    self.text_encoder = self.encode_text

    self.projection = ProjectionHead(embedding_dim)
    #self.classifier = torch.nn.Linear(1024, num_classes)
    # add a bottleneck
    self.classifier = torch.nn.Sequential(
      torch.nn.Linear(in_features, out_features, bias=bias),
      torch.nn.GELU(),
      torch.nn.Linear(out_features, out_features, bias=bias),
      torch.nn.Dropout(0.1),
      torch.nn.BatchNorm1d(1024),
    )

    def givemebox(self, img, bbox):
      x1 = int(bbox[0])
      y1 = int(bbox[1])
      x2 = int(bbox[2])
      y2 = int(bbox[3])
      t = torch.stack((img[0][0][y1:y2, x1:x2], img[0][1][y1:y2, x1:x2], img[0][2][y1:y2, x1:x2]))
      t = to_pil(t)
      t = preprocess(t).unsqueeze(0).to(device)
      return t

  def encode_text(self, text):
      x = model.token_embedding(text).float()
      x = x + model.positional_embedding.float()
      x = x.permute(1, 0, 2)  # NLD -> LND
      #x = x.type(torch.float16).to(device)
      x = model.transformer(x).float()
      x = x.permute(1, 0, 2)  # LND -> NLD
      x = model.ln_final(x).float()

      # x.shape = [batch_size, n_ctx, transformer.width]
      # take features from the eot embedding (eot_token is the highest number in each sequence)
      x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ model.text_projection

      return x


  def forward(self, batch):
    similar = []
    '''(batch['image'].shape)
    clip_image = to_pil(np.squeeze(batch['image']))

    im_features = []
    results = self.detector(clip_image, size=640)
    results = non_max_suppression(results, 0.2, 0.2)
    for det in results:
      for *xyxy, conf, cls in det:
        try:
          t = self.givemebox(clip_image, xyxy)
        except Exception as e:
          continue
        with torch.no_grad():
            image_features = self.clip_model.encode_image(t)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        im_features.append(image_features)
    im_features = torch.stack(im_features)'''
    image_features = self.classifier(batch['image'])
    captions = batch['captions'].to(device)
    y = self.text_encoder(captions)
    text_features = self.classifier(y)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    #target_len = y.size(0) - x.size(0)
    #x = F.pad(x, (0,0,0,target_len), value=0)
    #similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    #values, indices = similarity[0].topk(1)
    '''image_features_similarity = image_features @ image_features.T
    text_features_similarity = text_features @ text_features.T


    targets = F.softmax(
      (image_features_similarity + text_features_similarity) / 2 * temperature, dim=-1
    )


    outputs =  (image_features_similarity @ text_features_similarity.T) / temperature'''

    similarity = (100.0 * image_features @ text_features.T).softmax(dim=0)
    top_values, top_indices = similarity.topk(1, dim=0)

    return results[top_indices]


In [19]:
import torch
import torch.nn as nn

class IoULoss(nn.Module):
    def __init__(self):
        super(IoULoss, self).__init__()

    def forward(self, pred_boxes, target_boxes):
        batch_size, num_boxes = pred_boxes.size()

        iou = calculate_iou(pred_boxes, target_boxes)
        loss = 1 - iou
        return loss.mean()


In [20]:
def get_cost_function():
  loss = IoULoss()
  return loss

In [21]:
def get_optimizer(model, lr, wd, momentum):
  optimizer = torch.optim.SGD([
      {'params': model.parameters(), 'lr': lr}
  ], lr=lr / 10, weight_decay=wd, momentum=momentum)

  return optimizer

In [22]:
def pad_sequence(batch):
  return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0)
def my_collate_fn(batch):

  return {
      'image': torch.stack([x['image'] for x in batch]),
      'captions': torch.cat([x['captions'] for x in batch]),
      'bbox' : torch.stack([x['bbox'] for x in batch])
      }


def get_data(data_dir, batch_size, transform=True, target_transform=False, test_batch_size=15):


    if transform:
        # convert the PIL images to Tensors
        transform = preprocess
    else:
          # prepare data transformations and then combine them sequentially
        transform = T.Compose([torchvision.transforms.ToTensor()])
    if target_transform:
        target_transform = T.Compose([
                                 lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)])
    else:
        target_transform = None

  # load data
    dataset = preProcess_datasets(data_dir, train=True)
    full_training_data = RefCOCOgDataset(dataset=dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)

    test_dataset = preProcess_datasets(data_dir, train=False)
    test_data = RefCOCOgDataset(dataset=test_dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)



    evens = list(range(0, len(full_training_data), 2))
    #training_data2 = torch.utils.data.Subset(full_training_data, evens)
    training_data2 = torch.utils.data.Subset(full_training_data, range(50))
    test_data2 = torch.utils.data.Subset(test_data, range(50))

  # create train and validation splits
    num_samples = len(training_data2)
    training_samples = int(num_samples * 0.5 + 1)
    validation_samples = num_samples - training_samples

    training_data, validation_data = torch.utils.data.random_split(training_data2, [training_samples, validation_samples])

  # initialize dataloaders_collate
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=0, collate_fn=my_collate_fn)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=0, collate_fn=my_collate_fn)
    #test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_data2, test_batch_size, shuffle=False, num_workers=0, collate_fn=my_collate_fn)

    print(len(training_data))

    print(len(validation_data))
    print(len(test_data))

    # pre-train model on zero-shot transfer learning
    #test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z)


    # evaluate accuracy on zero-shot learning

    #print("Test accuracy {:.2f}".format(test_accuracy))

    return train_loader, val_loader, test_loader


In [23]:
# tensorboard logging utilities
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [24]:
modified_model = CustomCLIP(num_classes=10).to(device)

In [25]:
# main funcition
def main(
      root='/content/dataset/refcocog/',
      data_dir='dataset/refcocog',
      batch_size=2,
      num_classes=10,
      learning_rate=0.000000000001,
      weight_decay=0.000001,
      momentum=0.9,
      #epochs=10,
      epochs=3
    ):
  writer = SummaryWriter(log_dir="runs/exp1")
  global test_loader
  # train clip on zero-shot learning and instantiates dataloaders
  train_loader, val_loader, test_loader = get_data(data_dir=data_dir, batch_size=batch_size, transform=True, target_transform=True, test_batch_size=2)
  #train_loader, test_loader = get_data(data_dir=data_dir, batch_size=batch_size, transform=True, target_transform=True, test_batch_size=32)
  #dataset = get_data(data_dir=data_dir, batch_size=batch_size, transform=True, test_batch_size=None)

  # instantiate the network and move it to the chosen device (GPU)


  # instantiate the optimizer
  optimizer = get_optimizer(modified_model, learning_rate, weight_decay, momentum)

  # define the cost function
  cost_function = get_cost_function()

  # evaluate accuracy of modified model on zero-shot learning
  #test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z).to(device)

  #print(cosine_similarity(images_z, texts_z))
  #print("Test accuracy {:.2f}".format(test_accuracy))

  # computes evaluation results before training
  print('Before training:')
  train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
  val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
  test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

  # log to TensorBoard
  log_values(writer, -1, train_loss, train_accuracy, "train")
  log_values(writer, -1, val_loss, val_accuracy, "validation")
  log_values(writer, -1, test_loss, test_accuracy, "test")

  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # for each epoch, train the network and then compute evaluation results
  for e in range(epochs):

    train_loss, train_accuracy = training_step(modified_model, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)

    # logs to TensorBoard
    log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

    # compute final evaluation results
    print('After training:')
    train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
    test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

    # log to TensorBoard
    log_values(writer, epochs, train_loss, train_accuracy, "train")
    log_values(writer, epochs, val_loss, val_accuracy, "validation")
    log_values(writer, epochs, test_loss, test_accuracy, "test")

    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')

  # closes the logger
  writer.close()


In [26]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<1024>"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [27]:
!pip install scikit-learn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score

def calculate_accuracy(output, target):


  f1_scr = f1_score(target, output, average='weighted')

  cm = confusion_matrix(target, output)
  TP = cm[0][0]
  FP = cm[0][1]
  FN = cm[1][0]
  TN = cm[1][1]

  #accuracy = np.sum(np.diagonal(cm)) / np.sum(cm)
  accuracy = (TP+TN)/(TP+FP+FN+TN)
  return accuracy

In [189]:
main()

ValueError: ignored

In [ ]:
#!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
#im = cv2.imread("./input.jpg")
def draw_bbx(im):
  #!python yolov5/classify/predict.py --weights yolov5s.pt --source im --view-img
  results = detector(im)
  return results

In [ ]:
def get_img_embeddings(loader, model):

  test_image_embeddings = []

  for idx, batch in enumerate(loader):
    batch['image'] = batch['image'].to(device)
    image_features = model.encoder(batch['image'])
    image_embeddings = model.projection(image_features)
    test_image_embeddings.append(image_embeddings)

  test_image_embeddings = torch.cat(test_image_embeddings)

  return test_image_embeddings


def inference(model, query, image_filenames, n):

    model.eval()
    global image_embeddings

    tokenized_query = clip.tokenize([query]).to(device)
    '''batch = {
        key: torch.tensor(values).to(device)
        for key, values in encoded_query.items()
    }'''


    with torch.no_grad():

      image_embeddings = get_img_embeddings(test_loader, model)
      text_features = model.text_encoder(tokenized_query)
      text_embeddings = model.projection(text_features)

      image_embeddings_n = F.normalize(image_embeddings, p=2, dim=-1)
      text_embeddings_n = F.normalize(text_embeddings, p=2, dim=-1)
      dot_similarity = text_embeddings_n @ image_embeddings_n.T

      #print(dot_similarity.squeeze(0))
      val, index = torch.topk(dot_similarity.squeeze(0), n)
      found_match = image_filenames[index]

      plt.figure(1)
    #for match, ax in zip(matches, axes.flatten()):
      image = cv2.imread(f"{'dataset/refcocog/images'}/{found_match['file_name']}")
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = draw_bbx(image)
      results.show()
      '''imgplot = plt.imshow(image)
      plt.axis('off')
      plt.show()
      #draw_bbx(image)'''





In [ ]:
inference(modified_model,
             query="tree",
             image_filenames=data['images'], n=1)


In [ ]:
from torchvision import transforms
to_pil = transforms.ToPILImage()
def baseline(x):
    image = x[0]
    captions = x[1]
    result = yolo(image)
    bboxes = result.crop(save=False)
    im_features = []
    for bbox in bboxes:
        image_input = preprocess(Image.fromarray(bbox['im'])).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image_input)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        im_features.append(image_features)
    if len(bboxes) == 0:
      return 0, torch.zeros(4)
    text_inputs = captions.unsqueeze(0).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)

    text_features /= text_features.norm(dim=-1, keepdim=True)
    im_features = torch.stack(im_features)
    similarity = (100.0 * im_features @ text_features.T).softmax(dim=0)
    top_values, top_indices = similarity.topk(1, dim=0)
    return top_values.item(), bboxes[top_indices]['box']

In [ ]:
pred_boxes = []
true_boxes = []

for idx, x in enumerate(dataset):
  print(f"\nTop predictions for {dataset.descriptions[idx][0]['raw']}:-------------------------\n")
  conf, box = baseline(x)
  print("target:", x[2])
  print("confidence: ", 100*conf)
  print("box: ", box)
  if conf != 0:
    x1,y1, x2, y2 = box
    pred_boxes.append([x1.item(), y1.item(), x2.item(), y2.item()])
    true_boxes.append(x[2].tolist())